# Proyecto: Clasificación de tópicos de interés

## Exploración de datos


CC5113 - Aprendizaje Automático Bayesiano

Profesor: Pablo Guerrero

Autor: Martín Cornejo

## Paquetes necesarios

In [35]:
import numpy as np
import matplotlib.pyplot as plt
import pdb
import itertools
import operator

import pandas as pd

from nltk.stem.snowball import SpanishStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [23]:
datos=pd.read_csv('data_format.csv')
print(datos.shape)
datos.head()

(185, 2)


,Texto,Interes
0,"Ojalá obliguen a Piñera a cerrar Punta Peuco, ...",False
1,Piñera para crear base de apoyo moderada a su ...,True
2,@CNNChile MEMORIA 2014 Adimark: Piñera termina...,True
3,PPK y Piñera en privado habrían conversado alg...,False
4,Bachelet entregará el gobierno de Chile a Piñera,True


## Limpiando strings

### Reemplazar tildes, caracteres especiales, todo a minúsculas

In [4]:
strings = datos.ix[:,0]

def formatear(strings):
    tildes = ['á','é','í','ó','ú']
    vocales = ['a','e','i','o','u']

    # tildes
    for idx, vocal in enumerate(vocales):
        strings = strings.str.replace(tildes[idx],vocal)

    # caracteres especiales menos la ñ
    strings = strings.str.replace('[^a-zñA-Z ]', "")

    # todo a minusculas
    strings = pd.Series(list(map(lambda x: x.lower(), strings)))
    
    return strings

def oracionToStrArr(strings):
    strings_arr = list(map(lambda x: x.split(), strings))
    #pdb.set_trace()
    strings_arr = list(itertools.chain.from_iterable(strings_arr))    
    return strings_arr

print(formatear(strings).head())
formated_array_data = oracionToStrArr(formatear(strings))

0    ojala obliguen a piñera a cerrar punta peuco e...
1    piñera para crear base de apoyo moderada a su ...
2    cnnchile memoria  adimark piñera termina su go...
3    ppk y piñera en privado habrian conversado alg...
4     bachelet entregara el gobierno de chile a piñera
dtype: object


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


## Funciones auxiliares (conteo y ordenar repetidas)

In [5]:
def most_common(str_arr):
  # get an iterable of (item, iterable) pairs
  SL = sorted((x, i) for i, x in enumerate(str_arr))
  list_pairs = []
  #print('SL:', SL)
  groups = itertools.groupby(SL, key=operator.itemgetter(0))
    
  # auxiliary function to get "quality" for an item
  def _auxfun(g):
    item, iterable = g
    count = 0
    min_index = len(str_arr)
    for _, where in iterable:
      count += 1
      min_index = min(min_index, where)
    list_pairs.append((item, count))
    #print('item %r, count %r, minind %r' % (item, count, min_index))
    return count, -min_index

  return max(groups, key=_auxfun)[0], list_pairs

def aplicar_ordenar_str_arr(func, str_arr):
    arr = list(map(func, str_arr))
    common, pairs = most_common(arr)
    pares_filtrados = list(filter(lambda x: len(x[0]) > 3, pairs))
    common_sorted = sorted(pares_filtrados, key=lambda tup: tup[1], reverse=True)
    return common_sorted

## Separando data por clases

In [6]:
datos_interes = datos[datos.Interes == True]
#print(datos_interes.head())
str_interes = datos_interes.ix[:,0]

datos_no_interes = datos[datos.Interes == False]
#print(datos_no_interes.head())
str_no_interes = datos_no_interes.ix[:,0]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


## Stemming por clases

In [7]:
stemmer_es = lambda x: SpanishStemmer().stem(x)

stem_interes_ordenado = pd.Series(aplicar_ordenar_str_arr(stemmer_es,oracionToStrArr(formatear(str_interes))))
stem_no_interes_ordenado = pd.Series(aplicar_ordenar_str_arr(stemmer_es,oracionToStrArr(formatear(str_no_interes))))

stem_interes_ordenado

0           (piñer, 80)
1       (president, 27)
2         (sebasti, 23)
3            (chil, 21)
4        (bachelet, 20)
5         (gobiern, 20)
6             (mand, 9)
7             (este, 8)
8          (reunion, 8)
9             (esta, 7)
10           (activ, 6)
11            (asum, 6)
12           (cambi, 6)
13          (doming, 6)
14           (macri, 6)
15           (mañan, 6)
16            (nuev, 6)
17            (pais, 6)
18        (asuncion, 5)
19           (cierr, 5)
20          (derech, 5)
21            (lleg, 5)
22           (polit, 5)
23            (tien, 5)
24            (ahor, 4)
25            (cerr, 4)
26          (chilen, 4)
27          (gobern, 4)
28           (mejor, 4)
29         (oficial, 4)
             ...       
425        (suramer, 1)
426          (tambi, 1)
427    (tempranisim, 1)
428          (tendr, 1)
429           (teng, 1)
430          (tiend, 1)
431          (titer, 1)
432         (todavi, 1)
433           (trag, 1)
434         (tramit, 1)
435           (t

In [8]:
stem_no_interes_ordenado

0         (piñer, 72)
1          (esta, 10)
2           (bien, 7)
3        (sebasti, 7)
4          (cuand, 6)
5           (peuc, 6)
6           (punt, 6)
7           (algo, 5)
8           (cerr, 5)
9           (chil, 5)
10         (culia, 5)
11         (quier, 5)
12          (ahor, 4)
13         (cierr, 4)
14          (pier, 4)
15     (president, 4)
16       (academi, 3)
17      (bachelet, 3)
18        (derech, 3)
19      (diplomat, 3)
20          (este, 3)
21          (falt, 3)
22          (gent, 3)
23       (gobiern, 3)
24        (ladron, 3)
25          (lleg, 3)
26         (minim, 3)
27          (mism, 3)
28         (penal, 3)
29       (reajust, 3)
            ...      
420         (suci, 1)
421     (suddenly, 1)
422         (suel, 1)
423        (sueld, 1)
424        (suert, 1)
425       (tampoc, 1)
426         (tant, 1)
427       (termin, 1)
428         (tond, 1)
429         (toqu, 1)
430       (tortur, 1)
431        (total, 1)
432      (traidor, 1)
433       (travaj, 1)
434       

## Lematización

In [9]:
def create_lemma_dict(filename):
   with open(filename, 'r') as document:
       lemma_dict = {}
       for line in document:
           if line.strip():  # avoid empty lines
               value, key = line.split(None, 1) # 'None' means 'all whitespace', which is the default
               key = key.rstrip() # rstrip() to get rid of \r and \n
               lemma_dict[key] = value # adding the flections as keys to the dict
               lemma_dict[value] = value # adding also the base word as a key
   return lemma_dict

def query_word(lemma_dict):
   word = input("\nDame una palabra en español -> ")
   try:
      lemma = lemma_dict[word]
      print("__your happy lemma is__: {}".format(lemma))
   except KeyError:
      print("This word is not in the dictionary!")
   return query_word(lemma_dict)

def lemmatiser(dict):
    def lookup(word):
        try:
            lemma = dict[word]
        except:
            lemma = word
        
        return lemma
    
    return lookup

## Lematizando por clases

In [10]:
resource_file = 'lemmatization-es.txt'
lemmatiser_es = lemmatiser(create_lemma_dict(resource_file))

#pdb.set_trace()

datos_interes = datos[datos.Interes == True]
#print(datos_interes.head())
str_interes = datos_interes.ix[:,0]

datos_no_interes = datos[datos.Interes == False]
#print(datos_no_interes.head())
str_no_interes = datos_no_interes.ix[:,0]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if sys.path[0] == '':


In [11]:
lema_interes_ordenado = pd.Series(aplicar_ordenar_str_arr(lemmatiser_es, oracionToStrArr(formatear(str_interes))))
lema_no_interes_ordenado = pd.Series(aplicar_ordenar_str_arr(lemmatiser_es, oracionToStrArr(formatear(str_no_interes))))

#lema_interes_ordenado

In [12]:
#lema_no_interes_ordenado

## Tokenizing usando sklearn

In [13]:
stop_words_es = np.genfromtxt('stop_words_es.txt', dtype='str')
stop_words_es = formatear(pd.Series(stop_words_es))
stop_words_es = list(map(lambda x: x, stop_words_es))

In [14]:
tf_vectorizer = CountVectorizer(min_df=2, max_features=50, stop_words=stop_words_es)
data = formatear(strings)
dtm_tf = tf_vectorizer.fit_transform(data)
#pdb.set_trace()
#True

## Aplicando Latent Dirichlet Allocation

In [15]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        #pdb.set_trace()
        message += " ".join([feature_names[i] + " " + str(int(topic[i])) + " "
                             for i in topic.argsort()[:-n_top_words - 1:-1]])        
        print(message)
    print()

In [16]:
lda = LatentDirichletAllocation(n_components=2, max_iter=100,
                                learning_method='online',
                                learning_offset=10,
                                batch_size=10,
                                random_state=1)

In [17]:
n_top_words = 6
lda.fit(dtm_tf)
print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)


Topics in LDA model:
Topic #0: piñera 56  sebastian 29  presidente 26  chile 26  mando 9  reunion 8 
Topic #1: piñera 96  gobierno 26  bachelet 18  peuco 10  punta 9  hace 8 



In [18]:
pyLDAvis.sklearn.prepare(lda, dtm_tf, tf_vectorizer)

/home/martin/.local/lib/python3.5/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x    y
topic                                           
0      50.769091        1       1  0.155005  0.0
1      49.230909        1       2 -0.155005  0.0, topic_info=     Category        Freq              Term       Total  loglift  logprob
term                                                                     
43    Default   28.000000         sebastian   28.000000  30.0000  30.0000
39    Default   25.000000        presidente   25.000000  29.0000  29.0000
8     Default   25.000000             chile   25.000000  28.0000  28.0000
18    Default   23.000000          gobierno   23.000000  27.0000  27.0000
33    Default    9.000000             peuco    9.000000  26.0000  26.0000
27    Default    9.000000             mando    9.000000  25.0000  25.0000
35    Default  137.000000            piñera  137.000000  24.0000  24.0000
41    Default    8.000000             punta    8.000000  23.0000  23.0000
4     Default   20.000000          bachelet   20.000000  22.0000  22.0000
42    Default    8.000000           reunion    8.000000  21.0000  21.0000
19    Default    7.000000              hace    7.000000  20.0000  20.0000
10    Default    6.000000            cierre    6.000000  19.0000  19.0000
24    Default    7.000000             macri    7.000000  18.0000  18.0000
5     Default    6.000000            cambio    6.000000  17.0000  17.0000
32    Default    6.000000              pais    6.000000  16.0000  16.0000
14    Default    6.000000           domingo    6.000000  15.0000  15.0000
29    Default    8.000000            mañana    8.000000  14.0000  14.0000
31    Default    5.000000             nuevo    5.000000  13.0000  13.0000
13    Default    5.000000                da    5.000000  12.0000  12.0000
36    Default    5.000000              pone    5.000000  11.0000  11.0000
2     Default    5.000000          asuncion    5.000000  10.0000  10.0000
48    Default    5.000000             todos    5.000000   9.0000   9.0000
25    Default    5.000000               mal    5.000000   8.0000   8.0000
47    Default    5.000000           tiempos    5.000000   7.0000   7.0000
30    Default    4.000000             mejor    4.000000   6.0000   6.0000
23    Default    4.000000             llego    4.000000   5.0000   5.0000
9     Default    4.000000           chileno    4.000000   4.0000   4.0000
1     Default    4.000000             asume    4.000000   3.0000   3.0000
6     Default    4.000000            carlos    4.000000   2.0000   2.0000
21    Default    4.000000              juan    4.000000   1.0000   1.0000
...       ...         ...               ...         ...      ...      ...
18     Topic2   23.364347          gobierno   23.904409   0.6858  -2.3527
33     Topic2    8.905248             peuco    9.396847   0.6549  -3.3173
41     Topic2    8.058564             punta    8.551307   0.6493  -3.4172
19     Topic2    7.409589              hace    7.890552   0.6458  -3.5012
10     Topic2    6.347505            cierre    6.828853   0.6356  -3.6559
36     Topic2    4.654431              pone    5.154417   0.6066  -3.9661
31     Topic2    5.339425             nuevo    5.961254   0.5985  -3.8288
48     Topic2    4.733637             todos    5.285114   0.5984  -3.9493
28     Topic2    3.874908             manos    4.355324   0.5918  -4.1494
7      Topic2    3.809053            cerrar    4.287951   0.5902  -4.1666
16     Topic2    3.810678              esto    4.293129   0.5894  -4.1662
11     Topic2    3.764700  cierrepuntapeuco    4.244105   0.5888  -4.1783
12     Topic2    3.805472            culiao    4.299275   0.5866  -4.1675
25     Topic2    4.608703               mal    5.206867   0.5866  -3.9760
3      Topic2    3.759739              años    4.250529   0.5860  -4.1796
22     Topic2    3.864842               les    4.377714   0.5840  -4.1520
46     Topic2    3.740806           termina    4.251969   0.5806  -4.1847
47     Topic2    4.584058           tiempos    5.218402   0.5

## Obteniendo las frecuencias

In [30]:
X_train, X_test, y_train, y_test = train_test_split(datos.ix[:,0], datos.ix[:,1], test_size = 0.25, random_state = 4)

tf_vectorizer = CountVectorizer(stop_words=stop_words_es)
dtm_tf = tf_vectorizer.fit_transform(formatear(X_train))
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(dtm_tf)
X_train_tfidf.shape

(138, 1057)

## Entrenando con Naive Bayes

In [31]:
clf = BernoulliNB().fit(X_train_tfidf, y_train)

## Predicción del modelo

In [32]:
# new_tweets = ['Viva Piñera', 'Creo que Piñera no tendrá buen desempeño en su nuevo gobierno', 'Piñera y Bachelet lideran gobiernos', 'Piñera y puntapeuco no se rinden']

# Se extraen las características de los nuevos tweets
dtm_tf_test = tf_vectorizer.transform(formatear(X_test))
X_new_tfidf_test = tfidf_transformer.transform(dtm_tf_test)

predicted = clf.predict(X_new_tfidf_test)
#for tweet, prediction in zip(data_test, predicted):
#    print('%r => %s' % (tweet, str(prediction)))

np.mean(predicted == y_test)

0.6595744680851063

## Entrenando con SVM

In [37]:
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=1e-3, random_state=42,
                                            max_iter=5, tol=None))])
text_clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False))])

In [40]:
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

0.7659574468085106